## Notebook for collecting metadata, processing imaging data through suite2p and analysing trial responses

In [4]:
%run ./rob_setup_notebook.ipynb

import utils.interareal_analysis as ia

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload

Suite2p path: ['/home/rlees/Documents/Code/suite2p/suite2p']
QNAP_path: /home/rlees/mnt/qnap 
QNAP_data_path /home/rlees/mnt/qnap/Data 
Pickle folder: /home/rlees/mnt/qnap/pkl_files 
Master pickle: /home/rlees/mnt/qnap/master_pkl/master_obj.pkl 
Figure save path: /home/rlees/mnt/qnap/Analysis/Plots 
STA movie save path: /home/rlees/mnt/qnap/Analysis/STA_movies 
S2 borders path: /home/rlees/mnt/qnap/Analysis/S2_borders


### Define sessions to run s2p or analysis on

In [5]:
ss_id = '1PgSXs6BVHe9dACWEydykucN74rirNfOY4Vr_AmLcCdY'
sheet_names = [
#                 '2019-06-11_J048',
#                 '2019-06-13_J052',
#                 '2019-06-25_RL048',
#                 '2019-07-15_RL052',
#                 '2019-08-14_J058',
#                 '2019-08-14_J059',
#                 '2019-08-15_RL055',
                '2019-11-14_J061',
                '2019-11-14_J063',
                '2019-11-15_J064',
                '2019-11-18_J065',
                '2020-02-27_RL075',
                '2020-03-03_RL074',
                '2020-03-05_RL070',
                '2020-03-05_RL072',
#                 '2020-03-12_RL070',
#                 '2020-03-16_RL074',
#                 '2020-03-19_RL075'
              ]

### Run s2p on defined sessions (commented-out to not overwrite data by accident)

In [ ]:
# for sheet_name in sheet_names:
    
#     # Initialise the class
#     ses_obj = ia.interarealProcessing(ss_id, sheet_name, qnap_data_path)
    
#     # Run s2p on this experiment (if not already done)
#     ses_obj.s2pRun()
    
#     # Pickle the object output to save it for analysis
#     pkl_path = os.path.join(pkl_folder, sheet_name + '.pkl')
    
#     with open(pkl_path, 'wb') as f:
#         pickle.dump(ses_obj, f)

### Run sessions through analysis pipeline

In [10]:
trial_sig_calc = 'dff' # how to calculate trial significance ('dfsf' or 'dff')

for sheet_name in sheet_names:
    
    # Initialise the class
    ses_obj = ia.interarealProcessing(ss_id, sheet_name, qnap_data_path)
    
    # Get the range of frames that are attributed to each experiment (from the concatenated t-series)
    ses_obj.getFrameRanges()
    
    # Process the s2p data (after manual curation)
    ses_obj.photostim_r.s2pAnalysis(s2_borders_path, trial_sig_calc)
    ses_obj.photostim_s.s2pAnalysis(s2_borders_path, trial_sig_calc)
    ses_obj.whisker_stim.s2pAnalysis(s2_borders_path, trial_sig_calc)
    
    # Add sham photostim metadata
    ses_obj.addShamPhotostim()
    
    ses_obj.spont.s2pAnalysis(s2_borders_path, trial_sig_calc)
    
    ses_obj.whiskerTargets()
    
    # Pickle the object output to save it for analysis
    pkl_path = os.path.join(pkl_folder, sheet_name + '.pkl')
    
    with open(pkl_path, 'wb') as f:
        pickle.dump(ses_obj, f)        


Fetching paths and stim types for: 2019-11-14_J061

Umbrella folder: /home/rlees/mnt/qnap/Data/2019-11-14
['/home/rlees/mnt/qnap/Data/2019-11-14/2019-11-14_J061/2019-11-14_J061_t-002']
['/home/rlees/mnt/qnap/Data/2019-11-14/2019-11-14_J061/2019-11-14_J061_t-002.paq']
['/home/rlees/mnt/qnap/Data/2019-11-14/2019-11-14_J061/2019-11-14_J061_t-003']
['/home/rlees/mnt/qnap/Data/2019-11-14/2019-11-14_J061/2019-11-14_J061_t-006']
['/home/rlees/mnt/qnap/Data/2019-11-14/2019-11-14_J061/2019-11-14_J061_NAPARM/2019-11-14_J061_NAPARM_002']
['/home/rlees/mnt/qnap/Data/2019-11-14/2019-11-14_J061/2019-11-14_J061_t-006.paq']
['/home/rlees/mnt/qnap/Data/2019-11-14/2019-11-14_J061/2019-11-14_J061_t-007']
['/home/rlees/mnt/qnap/Data/2019-11-14/2019-11-14_J061/2019-11-14_J061_NAPARM/2019-11-14_J061_NAPARM_003']
['/home/rlees/mnt/qnap/Data/2019-11-14/2019-11-14_J061/2019-11-14_J061_t-007.paq']

Experimental info = (array(['/home/rlees/mnt/qnap/Data/2019-11-14/2019-11-14_J061/2019-11-14_J061_t-007',
       